In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Documents/repos/Dubins3D.jl/example`


In [2]:
include("../src/Dubins3D.jl")
using Revise
using .Dubins3D

In [3]:
using StaticArrays, LinearAlgebra

In [4]:
qi = [0.0, 0.0, 0.0, 0.0, 0.0]
qf = [1.0, 1.0, 1.0, 0.0, 0.0]
turning_radius = 1.0
pitch_limits = [-π/4, π/4]

@time maneuver = DubinsManeuver3D(qi, qf, turning_radius, pitch_limits)
result = Vector{DubinsManeuver2D}(undef, 2)
@time cc = Dubins3D.try_to_construct!(result, maneuver, 10.0)

  0.073047 seconds (381.87 k allocations: 18.879 MiB, 99.86% compilation time)
  0.000006 seconds (6 allocations: 832 bytes)


true

In [5]:
q2i = @SVector [0.0, 0.0, 0.0]
q2f = @SVector [1.0, 0.0, 0.0]
rhomin = 0.5
pitchmax = @SVector [-pi/4, pi/4]

@time mm = DubinsManeuver2D(q2i, q2f, rhomin=rhomin)
@time lsl = Dubins3D._LSL(mm)
@time rsr = Dubins3D._RSR(mm)
@time lsr = Dubins3D._LSR(mm, pitchmax)
@time rsl = Dubins3D._RSL(mm, pitchmax)

  0.003409 seconds (71 allocations: 3.438 KiB, 99.25% compilation time)
  0.000011 seconds (6 allocations: 256 bytes)
  0.000005 seconds (6 allocations: 256 bytes)
  0.000010 seconds (6 allocations: 256 bytes)
  0.000005 seconds (6 allocations: 256 bytes)


Main.Dubins3D.DubinsStruct(6.283185307179586, 2.0000000000000004, 6.283185307179586, 7.283185307179586, "RSL")

In [6]:
qvi = @SVector [0.0, 0.0, 0.0]
qvf = @SVector [1.0, 0.0, 0.0]
rhomin = 0.5
pitchlims = @SVector [-π/4, π/4]
@time v = Dubins3D.Vertical(qvi, qvf, rhomin, pitchlims)

  0.000009 seconds (19 allocations: 992 bytes)


DubinsManeuver2D([0.0, 0.0, 0.0], [1.0, 0.0, 0.0], 0.5, Main.Dubins3D.DubinsStruct(0.0, 2.0, 0.0, 1.0, "LSL"))

In [10]:
qi = [0.0, 0.0, 0.0, 0.0, 0.0]
qf = [10.0, 1.0, 1.0, 0.0, 0.0]
turning_radius = 1.0
pitch_limits = [-pi/4, pi/4]

@time maneuver = DubinsManeuver3D(qi, qf, turning_radius, pitch_limits)
@show maneuver
@time samples = compute_sampling(maneuver)

  0.000040 seconds (294 allocations: 39.078 KiB)
maneuver = DubinsManeuver3D([0.0, 0.0, 0.0, 0.0, 0.0], [10.0, 1.0, 1.0, 0.0, 0.0], 1.0, [-0.7853981633974483, 0.7853981633974483], DubinsManeuver2D[DubinsManeuver2D([0.0, 0.0, 0.0], [10.0, 1.0, 0.0], 1.4118213950851644, Main.Dubins3D.DubinsStruct(0.10110341798244019, 6.916506211909455, 0.10110341798244019, 10.050351386460907, "LSR")), DubinsManeuver2D([0.0, 0.0, 0.0], [10.050351386460907, 1.0, 0.0], 1.4166179307620148, Main.Dubins3D.DubinsStruct(0.10059061287730511, 6.928792475205033, 0.10059061287730511, 10.100448590641033, "LSR"))], 10.10044859064103)
  0.000059 seconds (3 allocations: 39.188 KiB)


1000-element Vector{SVector{5, Float64}}:
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [0.01011038689646604, 3.620189994633628e-5, 3.6079937355704135e-5, 0.007161297703334159, 0.0071371108117714134]
 [0.02021974033636972, 0.0001447983672936806, 0.000144317911578255, 0.014322230623522616, 0.014274221623542827]
 [0.030327027180043375, 0.00032576170805725046, 0.0003247084092270754, 0.021482433996001034, 0.02141133243531424]
 [0.04043121492143628, 0.0005790457772233524, 0.0005772422415467823, 0.028641543093366886, 0.028548443247085654]
 [0.050531272004492034, 0.0009045859961849199, 0.0009019065449350838, 0.03579919324395797, 0.035685554058857066]
 [0.060626168139008944, 0.0013022993771338051, 0.0012986847815979908, 0.04295501985042812, 0.04282266487062848]
 [0.07071487461581177, 0.0017720845543979127, 0.0017675567403921898, 0.05010865840831911, 0.0499597756823999]
 [0.08079636462106361, 0.002313821822700446, 0.0023084985378548856, 0.05725974452462777, 0.05709688649417131]
 [0.09086961354954742, 0.002927373

In [5]:
qi = [0.0, 0.0, 0.0, 0.0, 0.0]
qf = [10.0, 10.0, 10.0, 0.0, 0.0]

turning_radius = 10.0
pitch_limits = [-pi/4, pi/4]

maneuver = DubinsManeuver3D(qi, qf, turning_radius, pitch_limits)
samples = compute_sampling(maneuver)

path = reduce(hcat, samples)

using Plots
plot(path[1, :], path[2, :], path[3, :], label="Dubins Path")

for p in eachcol(path)
    if p[4] > π || p[4] < -π
        @show p
    end
end